In [77]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib.colors import Normalize 
import pandas as pd
import glob
import os
import re
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_EVEN
import csv
import math
import copy
from scipy.optimize import curve_fit
import scipy.stats as st
import sys

In [80]:
class Cookie:
    # クラス変数を設定する
    name = "クッキー"
    # 初期化メソッドでインスタンス変数を定義する
    def __init__(self, taste):
        self.taste = taste
    # メソッドを定義する
    def bake(self):
        print(self.taste, self.name, "が焼けました")

# インスタンスを生成する
cookie1 = Cookie("プレーン")
cookie2 = Cookie("チョコ")
cookie3 = Cookie("チョコチップ")

# インスタンスを指定してメソッドを呼び出す
cookie1.bake()
cookie2.bake()
cookie3.bake()


プレーン クッキー が焼けました
チョコ クッキー が焼けました
チョコチップ クッキー が焼けました


In [3]:
1000.00//3


333.0